<h1>Set 1</h1>
<h5>Challenge 1</h5>
<p>Convert hex to base64</p>

In [501]:
# converters 
import codecs
import base64

def str2bytes(string):
    return bytearray(string, 'utf8')

def hex2bytes(hexin):
    return bytearray.fromhex(hexin)

def base642bytes(base64in):
    return base64.b64decode(base64in.encode('utf-8'))

def bytes2str(bytesin):
    return bytesin.decode("utf8", errors='replace')
    
def bytes2hex(bytesin):
    return codecs.encode(bytesin, encoding='hex')

def bytes2base64(bytesin):
    return codecs.encode(bytesin, encoding='base64')

In [56]:
hexstring = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
correct = b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

In [63]:
bytes2base64(hex2bytes(hexstring)).strip() == correct

True

<h5>Challenge 2</h5>
<p>XOR</p>

In [90]:
first = '1c0111001f010100061a024b53535009181c'
second = '686974207468652062756c6c277320657965'
answer = b'746865206b696420646f6e277420706c6179'


def XOR(first, second):
    return bytes(a ^ b for (a, b) in zip(first, second))

result = bytes2hex(XOR(hex2bytes(first),hex2bytes(second)))

In [91]:
answer == result

True

<h5>Challenge 3</h5>
<p>Single-byte XOR decryption</p>

In [588]:
# word lists
word_list_data = {}
with open('englishwords.txt', 'r') as f:
    data = f.read()
    for idx, word in enumerate(data.split('\n')):
        word_list_data[word] = idx

In [589]:
# char list
char_freq_data = {}
with open('letterfreq.txt', 'r') as f:
    data = f.read()
    for line in data.split('\n'):
        elements = line.split(' ')
        char_freq_data[elements[0].lower()] = round(float(elements[-3][:-1]) / 100, 4)

In [590]:
tocrack = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
bytestring = hex2bytes(tocrack)

In [599]:
import collections

def evaluateString(string):
    # check char frequency
    size = len(string)
    string = string.lower()
    tmpstr = string
    for char in 'abcdefghijklmnopqrstuvwxyz':
        tmpstr = tmpstr.replace(char, '')
    charRatio = len(tmpstr) / size
    if charRatio > 0.5:
        return 0.0
    # check words frequency
    words = string.split(' ')
    lenwords = sum([len(word) for word in words if len(word) > 2])
    lenmatches = 0.
    for word in words:
        if word in word_list_data.keys():
            if len(word) > 2:
                lenmatches += len(word)
    
    return (lenmatches / lenwords) + (1 - charRatio)

In [607]:
likelyhood = {}
for char in range(256):
    deltas = [byte ^ char for byte in bytestring]
    tmp = bytes2str(bytearray(deltas))
    likelyhood[char] = evaluateString(tmp)
likelyhood

sorted_dict = collections.OrderedDict(sorted({likelyhood[value]:value for value in likelyhood.keys() if likelyhood[value] > 0.1}.items(), reverse=True))

print("Char:", chr(sorted_dict[next(iter(sorted_dict))]), "/",sorted_dict[next(iter(sorted_dict))] , "with a probability:", next(iter(sorted_dict)))
print("Message:", bytes2str(bytearray([byte ^ sorted_dict[next(iter(sorted_dict))] for byte in bytestring])))

Char: X / 88 with a probability: 1.6341176470588237
Message: Cooking MC's like a pound of bacon


<h5>Challenge 4</h5>
<p>Detect single-char XOR</p>

In [284]:
with open('1_4.txt', 'r') as f:
    msgs = f.read().split('\n')
msgs = [hex2bytes(i) for i in msgs]

In [313]:
bestprobability = 0.0
bestmsgidx = None
bestcharidx = None

for idx, msg in enumerate(msgs):
    likelyhood = {}
    for char in range(256):
        deltas = [byte ^ char for byte in msg]
        tmp = bytes2str(bytearray(deltas))
        likelyhood[char] = evaluateString(tmp)
        
    sorted_dict = {likelyhood[value]:value for value in likelyhood.keys() if likelyhood[value] > 0.1}
    if len(sorted_dict) >= 1:
        currentprob = next(iter(sorted_dict))
        if currentprob > bestprobability:
            bestprobability = currentprob
            bestmsgidx = idx
            bestcharidx = chr(sorted_dict[next(iter(sorted_dict))])
 
print('Msgidx:',bestmsgidx)
print('Probability:', bestprobability)
print('Char:',bestcharidx)
print('Msg:',bytes2str(bytearray([byte ^ ord(bestcharidx) for byte in msgs[bestmsgidx]])))

Msgidx: 170
Probability: 0.6521739130434783
Char: 5
Msg: Now that the party is jumping



<h5>Challenge 5</h5>
<p>Repeating key XOR</p>

In [461]:
stringIN = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"
stringOUT = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a2622632427276527""2a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"
key = 'ICE'
keyB = str2bytes(key)


def XOR_uneven(first, second):
    size = len(second)
    second = [second[i % size] for i in range(len(first))]
    return bytearray(bytes(a ^ b for (a, b) in zip(first, second)))


In [462]:
arrays = [str2bytes(s) for s in stringIN.split('\n')]
arrays = str2bytes(stringIN)
resultE = XOR_uneven(arrays, keyB)
resultD = bytes2hex(resultE)

In [469]:
resultD.decode('utf-8') == stringOUT

True

<h5>Challenge 6</h5>
<p>Break repeating key XOR</p>

In [552]:
def hammingDist(first, second):
    s = 0
    for i in range(len(first)):
        x = first[i] ^ second[i]
        bits = 0
        while (x > 0):
            bits += x & 1
            x >>= 1
        s += bits
    return s

In [554]:
hammingDist(b'this is a test', b'wokka wokka!!!')

37

In [614]:
a='test'
b='bong'
b = [b[i % 4] for i in range(len(b))]
#bytearray(bytes(a ^ b for (a, b) in zip(first, second)))
b


['b', 'o', 'n', 'g']

In [496]:
# params
possible_keysize = range(2,41)

In [511]:
data = ''
with open('1_6.txt', 'r') as f:
    data = f.read()

In [521]:
data = base642bytes(data)

In [574]:
from itertools import combinations


# find keysize
num_keysizes = 4

distances = {}

# find correct keysize
for keysize in possible_keysize:
    
    blocks = []
    for i in range(num_keysizes):
        start = i * keysize
        end = (i * keysize) + keysize
        blocks.append(data[start : end])
    
    pairs = combinations(blocks, 2)
    dist = 0.0
    
    for (first, second) in pairs:
        dist += hammingDist(first, second)
    
    dist /= 6
    dist /= keysize
    
    
    distances[dist] = keysize
    
ordered_distances = collections.OrderedDict(sorted(distances.items()))
keysizes = [ordered_distances[i] for i in list(ordered_distances.keys())[:5]]
keysizes

[29, 5, 2, 24, 7]

In [585]:
# make blocks of keysize
best_keysize = keysizes[0]

keysize_blocks = {i:[] for i in range(best_keysize)}

for idx, byte in enumerate(data):
    keysize_blocks[idx % best_keysize].append(byte)
    
keysize_blocks[0]

[29,
 54,
 60,
 55,
 56,
 116,
 58,
 53,
 116,
 38,
 59,
 116,
 94,
 58,
 55,
 49,
 57,
 57,
 59,
 61,
 53,
 34,
 53,
 58,
 59,
 116,
 36,
 32,
 101,
 48,
 60,
 51,
 58,
 116,
 53,
 58,
 115,
 116,
 116,
 49,
 33,
 13,
 116,
 60,
 54,
 59,
 122,
 59,
 44,
 53,
 53,
 116,
 49,
 50,
 116,
 45,
 51,
 49,
 45,
 59,
 53,
 38,
 116,
 94,
 60,
 48,
 94,
 59,
 116,
 49,
 116,
 54,
 55,
 116,
 116,
 58,
 115,
 33,
 49,
 59,
 53,
 58,
 32,
 49,
 38,
 53,
 53,
 45,
 59,
 116,
 45,
 61,
 59,
 94,
 54,
 32,
 48,
 55,
 120,
 39]

In [586]:
for block in keysize_blocks.keys():
    for char in range(256):
        deltas = [byte ^ char for byte in bytestring]
        tmp = bytes2str(bytearray(deltas))
        likelyhood[char] = evaluateString(tmp)

[29,
 54,
 60,
 55,
 56,
 116,
 58,
 53,
 116,
 38,
 59,
 116,
 94,
 58,
 55,
 49,
 57,
 57,
 59,
 61,
 53,
 34,
 53,
 58,
 59,
 116,
 36,
 32,
 101,
 48,
 60,
 51,
 58,
 116,
 53,
 58,
 115,
 116,
 116,
 49,
 33,
 13,
 116,
 60,
 54,
 59,
 122,
 59,
 44,
 53,
 53,
 116,
 49,
 50,
 116,
 45,
 51,
 49,
 45,
 59,
 53,
 38,
 116,
 94,
 60,
 48,
 94,
 59,
 116,
 49,
 116,
 54,
 55,
 116,
 116,
 58,
 115,
 33,
 49,
 59,
 53,
 58,
 32,
 49,
 38,
 53,
 53,
 45,
 59,
 116,
 45,
 61,
 59,
 94,
 54,
 32,
 48,
 55,
 120,
 39]

In [608]:
first = 'first' 
second = 'second'

In [609]:
len(first) == len(second)

False

In [610]:
first.type

AttributeError: 'str' object has no attribute 'type'

In [611]:
type(first)

str

In [612]:
type(b'first')

bytes

In [622]:
65 <= ord('a') <= 97

True

In [624]:
ord('z')

122

In [619]:
type('a')

str

In [620]:
type(65)

int

In [7]:
d = {}


In [8]:
if next(iter(d)):
    print(1)

StopIteration: 

In [13]:
import collections
d = collections.OrderedDict({1:1})

In [15]:
len(d.keys())

1